# データサイエンス100本ノック（構造化データ加工編） - R

## はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートとCSVファイルからのデータ読み込みを行います
- 利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（!マークに続けてOSコマンドを入力することで、任意のubuntu Linuxコマンドが入力可能）
- 名前、住所等はダミーデータであり、実在するものではありません

In [1]:
# require('RPostgreSQL')
require('tidyr')
require('dplyr')
require('stringr')
require('caret')
require('lubridate')
require('rsample')
require('unbalanced')

# host <- 'db'
# port <- Sys.getenv()["PG_PORT"]
# dbname <- Sys.getenv()["PG_DATABASE"]
# user <- Sys.getenv()["PG_USER"]
# password <- Sys.getenv()["PG_PASSWORD"]

# con <- dbConnect(PostgreSQL(), host=host, port=port, dbname=dbname, user=user, password=password)
# df_customer <- dbGetQuery(con,"SELECT * FROM customer")
# df_category <- dbGetQuery(con,"SELECT * FROM category")
# df_product <- dbGetQuery(con,"SELECT * FROM product")
# df_receipt <- dbGetQuery(con,"SELECT * FROM receipt")
# df_store <- dbGetQuery(con,"SELECT * FROM store")
# df_geocode <- dbGetQuery(con,"SELECT * FROM geocode")

Loading required package: tidyr
Warning message:
"package 'tidyr' was built under R version 3.6.3"Loading required package: dplyr
Warning message:
"package 'dplyr' was built under R version 3.6.3"
Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Loading required package: stringr
Warning message:
"package 'stringr' was built under R version 3.6.3"Loading required package: caret
Warning message:
"package 'caret' was built under R version 3.6.3"Loading required package: lattice
Loading required package: ggplot2
Warning message:
"package 'ggplot2' was built under R version 3.6.3"Loading required package: lubridate

Attaching package: 'lubridate'

The following object is masked from 'package:base':

    date

Loading required package: rsample
Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
"

In [2]:
df_customer <- read.csv("./data/customer_shift-jis.csv", encoding="shift-jis")
head(df_customer, n = 5)

customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0
CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C
CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0
CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2


In [3]:
# df_category <- read.csv("./data/category.csv", encoding="UTF-8")
df_category <- read.csv("./data/category.csv", encoding="shift-jis") # エラーがでたので文字コードを変更
head(df_category, n = 5)

category_major_cd,category_major_name,category_medium_cd,category_medium_name,category_small_cd,category_small_name
4,惣菜,401,御飯類,40101,弁当類
4,惣菜,401,御飯類,40102,寿司類
4,惣菜,402,佃煮類,40201,魚介佃煮類
4,惣菜,402,佃煮類,40202,海草佃煮類
4,惣菜,402,佃煮類,40203,野菜佃煮類


In [4]:
df_product <- read.csv("./data/product.csv", encoding="UTF-8")
head(df_product, n = 5)

X.U.FEFF.product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
P040101001,4,401,40101,198,149
P040101002,4,401,40101,218,164
P040101003,4,401,40101,230,173
P040101004,4,401,40101,248,186
P040101005,4,401,40101,268,201


In [5]:
df_receipt <- read.csv("./data/receipt.csv", encoding="UTF-8")
head(df_receipt, n = 5)

sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
20181103,1257206400,S14006,112,1,CS006214000001,P070305012,1,158
20181118,1258502400,S13008,1132,2,CS008415000097,P070701017,1,81
20170712,1215820800,S14028,1102,1,CS028414000014,P060101005,1,170
20190205,1265328000,S14042,1132,1,ZZ000000000000,P050301001,1,25
20180821,1250812800,S14025,1102,2,CS025415000050,P060102007,1,90


In [6]:
df_store <- read.csv("./data/store.csv", encoding="shift-jis") # 正常に読まなかったので文字コードを変換した
head(df_store, n = 5)

store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.1180,35.63559,1698
S13002,国分寺店,13,東京都,東京都国分寺市本多二丁目,トウキョウトコクブンジシホンダニチョウメ,042-123-4008,139.4802,35.70566,1735
S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732
S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495
S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679


In [7]:
df_geocode <- read.csv("./data/geocode.csv", encoding="UTF-8") # 時間がかかる, Python では一瞬で読めた
head(df_geocode, n = 5)

X.U.FEFF.postal_cd,prefecture,city,town,street,address,full_address,longitude,latitude
060-0000,"北海道,札幌市中央区",,,,北海道札幌市中央区,141.34103,43.05513,NA
064-0941,"北海道,札幌市中央区","旭ケ丘,",,"北海道札幌市中央区旭ケ丘,141.31972",43.04223,,NA,NA
060-0042,"北海道,札幌市中央区",大通西,,１丁目,北海道札幌市中央区大通西１丁目,141.35637,43.06102,NA
060-0042,"北海道,札幌市中央区",大通西,,２丁目,北海道札幌市中央区大通西２丁目,141.35445,43.06080,NA
060-0042,"北海道,札幌市中央区",大通西,,３丁目,北海道札幌市中央区大通西３丁目,141.35275,43.06086,NA


# 演習問題

### はじめに：パイプ演算子について  
パイプ演算子 %>% はRの標準演算子ではなく、dplyrをインストールしたら使えるようになります。  
https://qiita.com/zakkiiii/items/ecb7ddf55c7f1bfacecf  

R-4.1.0 からはパイプ演算子がRの標準装備になるようです。  
The native pipe  
When the {magrittr} package introduced the pipe to R in 2014, it was building upon similar syntax in Unix / linux scripting languages (bash) and functional programming languages (like F#).  
https://www.r-bloggers.com/2021/05/new-features-in-r-4-1-0/

---
> R-046: 顧客データフレーム（df_customer）の申し込み日（application_date）はYYYYMMDD形式の文字列型でデータを保有している。これを日付型（dateやdatetime）に変換し、顧客ID（customer_id）とともに抽出せよ。データは10件を抽出すれば良い。

In [8]:
df_tmp <- cbind(df_customer["customer_id"], 
                strptime(df_customer$application_date, '%Y%m%d'))
colnames(df_tmp) <- c("customer_id","application_date")
head(df_tmp, 10)

customer_id,application_date
CS021313000114,2015-09-05
CS037613000071,2015-04-14
CS031415000172,2015-05-29
CS028811000001,2016-01-15
CS001215000145,2017-06-05
CS020401000016,2015-02-25
CS015414000103,2015-07-22
CS029403000008,2015-05-15
CS015804000004,2015-06-07
CS033513000180,2015-07-28


In [9]:
# 特に難しいことはしていません。時刻の文字列データの変換だけがポイントです。
# YYMMDD の文字列データを strptime() で"2015-09-05 JST"のような形式に変換
strptime(df_customer$application_date, '%Y%m%d')

    [1] "2015-09-05 JST" "2015-04-14 JST" "2015-05-29 JST" "2016-01-15 JST"
    [5] "2017-06-05 JST" "2015-02-25 JST" "2015-07-22 JST" "2015-05-15 JST"
    [9] "2015-06-07 JST" "2015-07-28 JST" "2015-09-14 JST" "2015-08-04 JST"
   [13] "2015-02-28 JST" "2015-12-09 JST" "2015-11-01 JST" "2015-06-10 JST"
   [17] "2015-03-19 JST" "2015-01-12 JST" "2018-12-07 JST" "2015-04-17 JST"
   [21] "2015-11-01 JST" "2014-12-20 JST" "2014-10-24 JST" "2015-09-15 JST"
   [25] "2015-02-27 JST" "2017-08-24 JST" "2017-04-18 JST" "2016-08-12 JST"
   [29] "2015-01-22 JST" "2015-05-28 JST" "2015-07-13 JST" "2015-11-23 JST"
   [33] "2015-02-01 JST" "2015-07-08 JST" "2015-09-17 JST" "2015-02-16 JST"
   [37] "2015-11-21 JST" "2016-05-14 JST" "2017-06-04 JST" "2016-04-11 JST"
   [41] "2015-04-25 JST" "2015-06-23 JST" "2017-02-19 JST" "2015-03-26 JST"
   [45] "2015-07-12 JST" "2015-09-25 JST" "2017-07-12 JST" "2015-07-31 JST"
   [49] "2014-11-27 JST" "2016-03-06 JST" "2016-03-02 JST" "2017-08-21 JST"
   [53] "201

---
> R-047: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）はYYYYMMDD形式の数値型でデータを保有している。これを日付型（dateやdatetime）に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [10]:
# やっていることは R-046 と同じです。
df_tmp <- cbind(df_receipt[c('receipt_no', 'receipt_sub_no')], 
                strptime(as.character(df_receipt$sales_ymd), '%Y%m%d'))
colnames(df_tmp) <- c("receipt_no","receipt_sub_no", "sales_ymd")
head(df_tmp, 10)

receipt_no,receipt_sub_no,sales_ymd
112,1,2018-11-03
1132,2,2018-11-18
1102,1,2017-07-12
1132,1,2019-02-05
1102,2,2018-08-21
1112,1,2019-06-05
1102,2,2018-12-05
1102,1,2019-09-22
1112,2,2017-05-04
1102,1,2019-10-10


---
> R-048: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）は数値型のUNIX秒でデータを保有している。これを日付型（POSIXct）に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [11]:
# df_receipt の sales_epoch データを見てみましょう。
head(df_receipt$sales_epoc)

[1] 1257206400 1258502400 1215820800 1265328000 1250812800 1275696000

In [12]:
# これを日付型に変換するには、as.POSIXct() で処理します。
head(as.POSIXct(df_receipt$sales_epoch, origin="1970-01-01"))

[1] "2009-11-03 09:00:00 JST" "2009-11-18 09:00:00 JST"
[3] "2008-07-12 09:00:00 JST" "2010-02-05 09:00:00 JST"
[5] "2009-08-21 09:00:00 JST" "2010-06-05 09:00:00 JST"

In [13]:
df_tmp <- cbind(df_receipt[c('receipt_no', 'receipt_sub_no')], 
                as.POSIXct(df_receipt$sales_epoch, origin="1970-01-01"))
colnames(df_tmp) <- c("receipt_no","receipt_sub_no", "sales_ymd")
head(df_tmp, 10)

receipt_no,receipt_sub_no,sales_ymd
112,1,2009-11-03 09:00:00
1132,2,2009-11-18 09:00:00
1102,1,2008-07-12 09:00:00
1132,1,2010-02-05 09:00:00
1102,2,2009-08-21 09:00:00
1112,1,2010-06-05 09:00:00
1102,2,2009-12-05 09:00:00
1102,1,2010-09-22 09:00:00
1112,2,2008-05-04 09:00:00
1102,1,2010-10-10 09:00:00


---
> R-049: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（POSIXct）に変換し、"年"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [14]:
# 2009-11-03 09:00:00 の型式の文字列から最初の４文字を抽出すればよいのです。
head(substring("2009-11-03 09:00:00", 1, 4))

[1] "2009"

In [15]:
df_tmp <- cbind(df_receipt[c('receipt_no', 'receipt_sub_no')], 
                substring(as.POSIXct(df_receipt$sales_epoch, origin="1970-01-01"), 1, 4))
colnames(df_tmp) <- c("receipt_no","receipt_sub_no", "sales_ymd")
head(df_tmp, 10)

receipt_no,receipt_sub_no,sales_ymd
112,1,2009
1132,2,2009
1102,1,2008
1132,1,2010
1102,2,2009
1112,1,2010
1102,2,2009
1102,1,2010
1112,2,2008
1102,1,2010


---
> R-050: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（POSIXct）に変換し、"月"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。なお、"月"は0埋め2桁で取り出すこと。データは10件を抽出すれば良い。

In [16]:
# 2010-09-22 09:00:00 の型式の文字列から6から7を抽出すればよいのです。
head(substring("2010-09-22 09:00:00", 6, 7))

[1] "09"

In [17]:
df_tmp <- cbind(df_receipt[c('receipt_no', 'receipt_sub_no')], 
                substring(as.POSIXct(df_receipt$sales_epoch, origin="1970-01-01"), 6, 7))
colnames(df_tmp) <- c("receipt_no","receipt_sub_no", "sales_ymd")
head(df_tmp, 10)

receipt_no,receipt_sub_no,sales_ymd
112,1,11
1132,2,11
1102,1,07
1132,1,02
1102,2,08
1112,1,06
1102,2,12
1102,1,09
1112,2,05
1102,1,10
